# Train Music Transformer
Since Fri. Feb. 25th, 2022

Set up training in colab


## Setup


### Ipython



In [34]:
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Colab



In [35]:
import os
import sys


if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    ! pip3 install sty icecream music21 transformers datasets

    sys.path.append(os.path.join('drive', 'My Drive', 'Research', 'Music with NLP', 'Symbolic-Music-Generation'))


from musicnlp.util import *


print(PATH_BASE)
print(DIR_PROJ)
print(PKG_NM)



/Users/stefanh/Documents/UMich/Research/Music with NLP
Symbolic-Music-Generation
musicnlp


### code

In [36]:
import transformers
from icecream import ic

from musicnlp.util import *
from musicnlp.model import long_music_transformer as lmt


seed = config('random-seed')
transformers.set_seed(seed)



## Prep for training


In [37]:
# fnm = 'musicnlp music extraction, dnm=POP909, n=909, mode=melody, 2022-02-22 19-00-40'
fnm = 'musicnlp music extraction, dnm=POP909, n=909, mode=melody, 2022-02-25 20-59-06'


md_nm = 'debug'
# md_nm = 'debug-large'

n = 4
# n = None

mdl, tokenizer, dset_tr, trainer = lmt.get_all_setup(model_name=md_nm, dataset_name=fnm, n_sample=n, random_seed=seed)
ic(trainer.args.output_dir)

%load_ext tensorboard
%tensorboard --logdir tb_log



Loading cached processed dataset at /Users/stefanh/Documents/UMich/Research/Music with NLP/datasets/MNLP-Combined/hf_datasets/musicnlp music extraction, dnm=POP909, n=909, mode=melody, 2022-02-25 20-59-06/cache-f417996c5f50becf.arrow
Loading cached processed dataset at /Users/stefanh/Documents/UMich/Research/Music with NLP/datasets/MNLP-Combined/hf_datasets/musicnlp music extraction, dnm=POP909, n=909, mode=melody, 2022-02-25 20-59-06/cache-bdad8eedf3d98cb4.arrow
Loading cached processed dataset at /Users/stefanh/Documents/UMich/Research/Music with NLP/datasets/MNLP-Combined/hf_datasets/musicnlp music extraction, dnm=POP909, n=909, mode=melody, 2022-02-25 20-59-06/cache-e6c3ee9dc9d05a25.arrow
Loading cached processed dataset at /Users/stefanh/Documents/UMich/Research/Music with NLP/datasets/MNLP-Combined/hf_datasets/musicnlp music extraction, dnm=POP909, n=909, mode=melody, 2022-02-25 20-59-06/cache-47ffba45fbbacb63.arrow
Loading cached processed dataset at /Users/stefanh/Documents/UMi

Launching TensorBoard...

## Train


In [38]:
trainer.train()
trainer.save_model(trainer.args.output_dir)



2022-02-26 16:22:50| [LMTTransformer training]::on_train_begin::train.py:146, INFO - Training started with {#data: 909, model size: 8, learning rate: 0.0005, batch shape: (4, 8), #epochs: 8, #steps: 1816}
2022-02-26 16:22:54| [LMTTransformer training]::on_log::train.py:194, INFO - step=   1, epoch=  0.00, train_loss= 9.9020, learning_rate=5.00e-04
2022-02-26 16:22:57| [LMTTransformer training]::on_log::train.py:194, INFO - step=   2, epoch=  0.01, train_loss= 9.9006, learning_rate=5.00e-04
2022-02-26 16:23:00| [LMTTransformer training]::on_log::train.py:194, INFO - step=   3, epoch=  0.01, train_loss= 9.8790, learning_rate=5.00e-04
2022-02-26 16:23:03| [LMTTransformer training]::on_log::train.py:194, INFO - step=   4, epoch=  0.02, train_loss= 9.8650, learning_rate=5.00e-04
2022-02-26 16:23:06| [LMTTransformer training]::on_log::train.py:194, INFO - step=   5, epoch=  0.02, train_loss= 9.8417, learning_rate=5.00e-04
2022-02-26 16:23:10| [LMTTransformer training]::on_log::train.py:194, 

KeyboardInterrupt: 